In [1]:
import pyro
import dill
import os

In [2]:
# Load saved parameters

save_path = "./saved_guides/"

with open(os.path.join(save_path, "inferred_parameters2.pkl"), "rb") as f:
    inferred_parameters = dill.load(f)
print("Loaded inferred parameters from file.")

Loaded inferred parameters from file.


In [3]:
[*inferred_parameters.named_parameters()]

[('loc',
  Parameter containing:
  tensor([-0.3419, -1.8542, -0.9878, -1.0106,  1.2021,  1.2849,  2.2740, -1.5860,
           1.9740,  1.5441,  0.9700, -0.0286, -2.1124,  1.3206,  1.6324,  1.6865,
           0.2351,  0.1138,  0.2430,  0.1416, -0.1850, -0.0614, -0.3492,  0.0622,
          -0.1692,  0.8840,  0.0276, -0.5106, -0.1815, -0.0883,  0.0168, -0.1803,
           0.2660, -0.1332,  0.1727,  0.1170,  0.1163,  0.0087,  0.0054, -1.7854,
           1.9843], requires_grad=True)),
 ('scale_unconstrained',
  Parameter containing:
  tensor([ 0.3795, -0.5289,  0.3884, -1.6352, -1.0087,  0.3064, -0.0165, -0.0054,
           0.0226, -0.1191,  0.2943,  0.3943, -0.9570, -0.8787,  0.1069, -0.7997,
           0.4267,  0.4204,  0.4664,  0.4563,  0.4117,  0.4294,  0.4210,  0.4147,
           0.4345,  0.2505,  0.3819,  0.4337,  0.4461,  0.3777,  0.4550,  0.4119,
           0.4483,  0.4159,  0.4482,  0.4427,  0.4390,  0.3692,  0.4392, -2.2280,
          -0.0145], requires_grad=True))]